In [2]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.datasets import load_svmlight_file

from xgboost import XGBClassifier

In [15]:
data = pd.read_csv("dataset/training.csv", sep=",")
data = data.drop(data.columns[[6,7,8,10]], axis=1)
data = data.dropna()

# Sainty Check
print(data.shape)
data.head()

(793, 8)


,Fatigue,Soreness,Desire,Irritability,SleepHours,SleepQuality,USG,perf_class
0,4.0,4.0,5.0,4.0,8.75,5.0,1.019,0
1,5.0,4.0,6.0,4.0,8.00,4.0,1.009,1
2,3.0,6.0,6.0,5.0,6.25,1.0,1.021,1
3,4.0,4.0,4.0,4.0,9.25,4.0,1.009,0
4,5.0,4.0,6.0,4.0,9.00,4.0,1.009,2


In [18]:
# Split the dataset. 
X_train, X_test, y_train, y_test = train_test_split(data.loc[:,"Fatigue":"USG"], 
                                                    data["perf_class"], 
                                                    test_size = 0.2)
# Sainty Check
print(X_train.shape)

(634, 7)


In [19]:
model = XGBClassifier(objective = "binary:logistic")
model.fit(X_train, y_train)

y_pred = model.predict(X_test)
predictions = [round(value) for value in y_pred]

accuracy = accuracy_score(y_test, predictions) 
print("Accuracy: %.2f%%" % (accuracy * 100.0))

Accuracy: 42.14%


/Users/Michavillson/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
